The original implementation: https://github.com/MHersche/eegnet-based-embedded-bci

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import mne
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
from pathlib import Path

import torch
from torch import nn
from torch.utils.data import DataLoader

current_directory = Path().cwd()
parent_directory = current_directory.parent
sys.path.insert(0, str(parent_directory))

from modules.dataloader import CreateIDs, DataSet
from modules.architecture import EEGNet_lowpower
from training_loop import TrainingLoop

sns.set_style('whitegrid')

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
DOWNSAMPLING_FACTOR = 2
NUMBER_OF_CHANNELS = 64
N_SAMPLES_PER_RECORDING = 5
TASK_MODE = 'all_tasks'

# Model

* Ns - number of input samples in time domain
* Nch - number of EEG channels
* Ncl - number of classes
* Nf - filter size of first temporal filter
* Np - pooling length
</br></br>
* n - number of filters
* p - padding strategy

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Using {} device'.format(device))

Using cpu device


In [6]:
EEGNet = EEGNet_lowpower(task_mode=TASK_MODE)
if torch.cuda.device_count() > 1:
    print('Using {} GPUs'.format(torch.cuda.device_count()))
    EEGNet = nn.DataParallel(EEGNet)
else:
    print(f'Loading the model to {device}')
print(EEGNet)

Loading the model to cpu
EEGNet_lowpower(
  (temp_conv): TemporalConvolution(
    (conv1): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=same)
    (conv2): Conv2d(8, 8, kernel_size=(64, 1), stride=(1, 1), padding=same)
  )
  (batch_norm_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (depthwise_conv): DepthWiseConvolution(
    (conv1): Conv2d(8, 8, kernel_size=(64, 1), stride=(1, 1), padding=valid, groups=8)
    (conv2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), padding=valid)
  )
  (batch_norm_2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu_1): ELU(alpha=1.0)
  (avg_pool_1): AvgPool2d(kernel_size=(1, 8), stride=8, padding=0)
  (separable_conv): Conv2d(16, 16, kernel_size=(16, 1), stride=(1, 1), padding=same)
  (batch_norm_3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu_2): ELU(alpha=1.0)
  (avg_pool_2): AvgPool2d(kernel_size=(1, 8), stride=(1, 8)

In [7]:
X = torch.rand(1,1,64,480, device=device)
output = EEGNet(X)
output

tensor([[0.1414, 0.1920, 0.1864, 0.3120, 0.1682]], grad_fn=<SoftmaxBackward0>)

## Data loader

In [8]:
dataset_path = ''

train_ids, val_ids, test_ids = CreateIDs(dataset_path=dataset_path,
n_samples_per_recording=N_SAMPLES_PER_RECORDING, task_mode=TASK_MODE, split_ratio=(0.7,0.2,0.1)).create()

Found 109 patient's folders
Data will be split with ratio 70.0% train, 20.0% val, 10.0% test
Splitting data into train, val and test set according to recordings, val and test will use ceil int
Checking train IDs


  0%|          | 0/4895 [00:00<?, ?it/s]

Checking validation IDs


  0%|          | 0/1635 [00:00<?, ?it/s]

Checking test IDs


  0%|          | 0/1090 [00:00<?, ?it/s]

Created 4245 train, 1350 validation, and 945 test IDs


Dataset modes:
1. rest_unrest - predicts T0 vs rest (2 classes), uses all runs
2. left_right - predicts T0-T1-T2 (3 classes), uses runs 3,4,7,8,11,12, predicts left (T1) first or right first(T2)
3. upper_lower - predicts T0-T1-T2 (3 classes), uses runs 5,6,9,10,13,14
4. all tasks - predicts 5 classes (rest, left, right, both, feet)

In [9]:
training_dataset = DataSet(list_IDs=train_ids, dataset_path=dataset_path, sample_length=480, task_mode=TASK_MODE)
validation_dataset = DataSet(list_IDs=val_ids, dataset_path=dataset_path, sample_length=480, task_mode=TASK_MODE)
test_dataset = DataSet(list_IDs=test_ids, dataset_path=dataset_path, sample_length=480, task_mode=TASK_MODE)

training_data_loader = DataLoader(training_dataset, batch_size=10,shuffle=True)
validation_data_loader = DataLoader(training_dataset, batch_size=10,shuffle=True)
test_data_loader = DataLoader(training_dataset, batch_size=10,shuffle=True)

for x_train,y in training_data_loader:
    break

for x_test,y in test_data_loader:
    break
y

tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]])

# Training

In [10]:
save_path = ''

torch.cuda.empty_cache()
EEGNet = EEGNet_lowpower(task_mode=TASK_MODE).to(device)

#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(EEGNet.parameters(), lr=0.001)

training_loop = TrainingLoop(
    model=EEGNet,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    training_DataLoader=training_data_loader,
    epochs=2,
    save_path=save_path
)

training_losses, validation_losses, lr_rates = training_loop.run_training()

  0%|          | 0/2 [00:00<?, ?it/s]